# More Practice With SQL Queries - Lab

## Introduction

In this lesson, we'll run through some practice questions to refresh our knowledge of SQL Queries!

## Objectives

You will be able to:

- Practice your SQL knowledge

## Getting Started

As in previous labs, we'll make use of the `sqlite3` library as well as `pandas`. By combining them, we'll be able to write our queries as python strings, and make sure that the results are always returned as a pandas DataFrame. 

We'll start by loading both libraries and connecting to the database we'll be using for this lab, `data.sqlite`. You may remember this database from a previous lab. As a refresher, here's the ERD diagram for this database: 

<img src='images/Database-Schema.png'>

In the cell below:

* Import the necessary libraries `pandas` and `sqlite3`
* Establish a connection to the database `data.sqlite`
* Get the `cursor` from the connection and store it in the variable `c`.

In [1]:
import pandas as pd
import sqlite3

conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()

## Basic Queries

Now, let's review basic SQL queries. In the cell below:

* Write a query that gets the first name, last name, phone number, address, and credit limit for all customers in California with a credit limit greater than 25000.00. 

In [2]:
c.execute("""SELECT contactFirstName, contactLastName,
                    phone, addressLine1,
                    creditLimit
                    FROM customers 
                    WHERE state == 'CA' 
                          and creditLimit > 25000;
          """)
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
print(len(df))
df

10


,contactFirstName,contactLastName,phone,addressLine1,creditLimit
0,Susan,Nelson,4155551450,5677 Strong St.,210500.0
1,Julie,Murphy,6505555787,5557 North Pendale Street,64600.0
2,Juri,Hashimoto,6505556809,9408 Furth Circle,84600.0
3,Julie,Young,6265557265,78934 Hillside Dr.,90700.0
4,Valarie,Thompson,7605558146,361 Furth Circle,105000.0
5,Julie,Brown,6505551386,7734 Strong St.,105000.0
6,Brian,Chandler,2155554369,6047 Douglas Av.,57700.0
7,Sue,Frick,4085553659,3086 Ingle Ln.,77600.0
8,Steve,Thompson,3105553722,3675 Furth Circle,55400.0
9,Sue,Taylor,4155554312,2793 Furth Circle,60300.0


#### Expected Output

<img src='images/expected-output-1.png'>

## Aggregate Functions and GROUP BY

Next, write a query that get sthe average credit limit per state.

In [3]:
c.execute("""SELECT state, AVG(creditLimit) from customers GROUP BY state""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df

,state,AVG(creditLimit)
0,,61839.726027
1,BC,89950.000000
2,CA,83854.545455
3,CT,57350.000000
4,Co. Cork,0.000000
5,Isle of Wight,93900.000000
6,MA,70755.555556
7,NH,114200.000000
8,NJ,43000.000000
9,NSW,100550.000000


#### Expected Output

<img src='images/expected-output-2.png'>

## JOINs

Now, write a query that uses JOIN statements to get the customer name, customer number, order number, status, and quantity ordered. Print only the head of this DataFrame. 

In [4]:
c.execute("""SELECT c.customerName, c.customerNumber, o.orderNumber, o.status, od.quantityOrdered FROM Customers c JOIN Orders o 
ON c.customerNumber = o.customerNumber JOIN OrderDetails od ON od.orderNumber = o.orderNumber""")
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,orderNumber,status,quantityOrdered
0,Online Diecast Creations Co.,363,10100,Shipped,22
1,Online Diecast Creations Co.,363,10100,Shipped,30
2,Online Diecast Creations Co.,363,10100,Shipped,49
3,Online Diecast Creations Co.,363,10100,Shipped,50
4,"Blauer See Auto, Co.",128,10101,Shipped,25


#### Expected Output

<img src='images/joins.png'>

## HAVING and ORDER BY

Now, return the customerName, customrerNumber, productName, productCode and total number ordered for any product a customer has bought 10 or more of cumulatively. Sort the rows in descending order by the quantity ordered. 

**_Hint_**: For this one, you'll need to make use of HAVING, GROUP BY, and ORDER BY--make sure you get the order of them correct!

In [5]:
c.execute("""SELECT c.customerName, c.customerNumber, p.productName,
                    p.productCode, sum(od.quantityOrdered) as TotalOrdered
                    FROM Customers c JOIN Orders o 
                                     ON c.customerNumber = o.customerNumber
                                     JOIN OrderDetails od
                                     ON od.orderNumber = o.orderNumber
                                     JOIN Products p
                                     USING(productCode)
             GROUP BY c.customerNumber, productCode
             HAVING SUM(od.quantityOrdered) >= 10 
             ORDER BY TotalOrdered DESC"""
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,customerName,customerNumber,productName,productCode,TotalOrdered
0,Euro+ Shopping Channel,141,1992 Ferrari 360 Spider red,S18_3232,308
1,Euro+ Shopping Channel,141,1958 Chevy Corvette Limited Edition,S24_2840,245
2,Euro+ Shopping Channel,141,1970 Dodge Coronet,S24_1444,197
3,Euro+ Shopping Channel,141,1957 Chevy Pickup,S12_4473,183
4,Euro+ Shopping Channel,141,2002 Chevy Corvette,S24_3432,174


#### Expected Output

<img src='images/having_order.png'>

## Subqueries

Finally, get the first name, last name, employee number, and office code for employees from an office with less than 5 employees. 

In [6]:
c.execute("""select lastName, firstName, employeeNumber, officeCode
                    FROM employees
                    WHERE officeCode IN (SELECT officeCode 
                                                FROM offices 
                                                JOIN employees
                                                USING(officeCode)
                                                GROUP BY 1
                                                HAVING COUNT(employeeNumber) < 5
                                         );
          """
         )
df = pd.DataFrame(c.fetchall())
df.columns = [x[0] for x in c.description]
df.head()

,lastName,firstName,employeeNumber,officeCode
0,Patterson,William,1088,6
1,Firrelli,Julie,1188,2
2,Patterson,Steve,1216,2
3,Tseng,Foon Yue,1286,3
4,Vanauf,George,1323,3


#### Expected Output

<img src='images/expected-output-5.png'>

# Summary

In this lesson, we reviewed all the major concepts and keywords associated with SQL queries!